<p style="align: center;"><img align=center src="https://drive.google.com/uc?export=view&id=1I8kDikouqpH4hf7JBiSYAeNT2IO52T-T" width=600 height=480/></p>
<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Generative adversarial networks</b></h3>



В этом домашнем задании вы обучите GAN генерировать лица людей и посмотрите на то, как можно оценивать качество генерации

In [1]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
import torch.nn as nn
import cv2
from tqdm.notebook import tqdm
from torchvision.utils import save_image
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style='darkgrid', font_scale=1.2)

## Часть 1. Подготовка данных (0.5 балла)

В качестве обучающей выборки возьмем часть датасета [Flickr Faces](https://github.com/NVlabs/ffhq-dataset), который содержит изображения лиц людей в высоком разрешении (1024х1024). Оригинальный датасет очень большой, поэтому мы возьмем его часть. Скачать датасет можно [здесь](https://www.kaggle.com/datasets/tommykamaz/faces-dataset-small?resource=download-directory) и  [здесь](https://drive.google.com/file/d/1inyvLrN5wKBGCxQ4znMKBc64uL4uP_2x/view?usp=drive_link)

Давайте загрузим наши изображения. Напишите функцию, которая строит DataLoader для изображений, при этом меняя их размер до нужного значения (размер 1024 слишком большой, поэтому мы рекомендуем взять размер 128 либо немного больше)

In [13]:
!wget https://raw.githubusercontent.com/NVlabs/ffhq-dataset/master/download_ffhq.py

--2025-05-31 08:50:45--  https://raw.githubusercontent.com/NVlabs/ffhq-dataset/master/download_ffhq.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23829 (23K) [text/plain]
Saving to: ‘download_ffhq.py’

download_ffhq.py    100%[===================>]  23.27K  --.-KB/s    in 0.002s  

2025-05-31 08:50:45 (9.95 MB/s) - ‘download_ffhq.py’ saved [23829/23829]



In [14]:
!python download_ffhq.py --thumbs

|   0.00% done  1/2 files  0.00/0.25 GB   0.36 kB/s   ETA: ...     Traceback (most recent call last):
  File "/content/download_ffhq.py", line 445, in <module>
    run_cmdline(sys.argv)
  File "/content/download_ffhq.py", line 440, in run_cmdline
    run(**vars(args))
  File "/content/download_ffhq.py", line 387, in run
    download_files([json_spec, license_specs['json']], **download_kwargs)
  File "/content/download_ffhq.py", line 205, in download_files
    raise exc_info[1].with_traceback(exc_info[2])
  File "/content/download_ffhq.py", line 214, in _download_thread
    download_file(session, spec, stats, **download_kwargs)
  File "/content/download_ffhq.py", line 84, in download_file
    raise IOError('Incorrect file size', file_path)
OSError: [Errno Incorrect file size] ffhq-dataset-v2.json


In [16]:
# Create directory structure
!mkdir -p data/faces/all_faces

# Download sample faces (alternative if FFHQ download fails)
!wget -O data/faces.zip https://example.com/small_faces_dataset.zip  # Replace with actual URL
!unzip data/faces.zip -d data/faces/all_faces/
!rm data/faces.zip

--2025-05-31 08:53:16--  https://example.com/small_faces_dataset.zip
Resolving example.com (example.com)... 23.192.228.80, 23.192.228.84, 23.215.0.136, ...
Connecting to example.com (example.com)|23.192.228.80|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-05-31 08:53:16 ERROR 404: Not Found.

Archive:  data/faces.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of data/faces.zip or
        data/faces.zip.zip, and cannot find data/faces.zip.ZIP, period.


In [19]:
import os
import shutil
import requests
import zipfile
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt

def download_sample_faces():
    """Download and extract sample face images"""
    print("Downloading sample face images...")
    try:
        # Download sample dataset (5 face images)
        url = "https://github.com/NVlabs/ffhq-dataset/raw/master/thumbnails128x128.zip"
        os.makedirs('./temp', exist_ok=True)

        # Download
        response = requests.get(url, stream=True)
        with open('./temp/sample_faces.zip', 'wb') as f:
            shutil.copyfileobj(response.raw, f)

        # Extract
        with zipfile.ZipFile('./temp/sample_faces.zip', 'r') as zip_ref:
            zip_ref.extractall('./temp')

        # Create proper structure
        os.makedirs('./data/faces/all_faces', exist_ok=True)

        # Move first 5 images
        src_dir = './temp/thumbnails128x128'
        for img in os.listdir(src_dir)[:5]:
            shutil.move(f"{src_dir}/{img}", f"./data/faces/all_faces/{img}")

        # Clean up
        shutil.rmtree('./temp')
        print("Successfully downloaded 5 sample face images")
    except Exception as e:
        print(f"Download failed: {e}")
        print("Please manually create this structure:")
        print("./data/faces/all_faces/")
        print("And place at least one image file there")

def verify_dataset_structure():
    """Verify the dataset structure is correct"""
    data_dir = './data/faces'
    required_subdir = f"{data_dir}/all_faces"

    # Check if subdirectory exists and contains images
    if os.path.exists(required_subdir):
        valid_extensions = {'.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp'}
        files = os.listdir(required_subdir)
        image_files = [f for f in files if os.path.splitext(f)[1].lower() in valid_extensions]

        if image_files:
            print(f"Found {len(image_files)} images in {required_subdir}")
            return True

    # If structure is invalid
    print("Invalid dataset structure detected")
    print("Creating proper structure...")
    os.makedirs(required_subdir, exist_ok=True)
    return False

def get_dataloader(image_size=128, batch_size=64):
    """Create dataloader with robust error handling"""
    data_dir = './data/faces'

    # Verify or create dataset structure
    if not verify_dataset_structure():
        download_sample_faces()

    # Define transformations
    transform = tt.Compose([
        tt.Resize((image_size, image_size)),
        tt.ToTensor(),
        tt.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    try:
        dataset = ImageFolder(root=data_dir, transform=transform)
        print(f"Successfully loaded dataset with {len(dataset)} images")
        print(f"Images are in: {dataset.root}")
    except Exception as e:
        print("\nFinal error:", str(e))
        print("\nMANUAL SETUP REQUIRED:")
        print("1. Create this exact folder structure:")
        print("   ./data/faces/all_faces/")
        print("2. Place your images directly in the 'all_faces' folder")
        print("3. Supported formats: .jpg, .png, etc.")
        print("\nExample commands:")
        print("!mkdir -p ./data/faces/all_faces")
        print("!wget -P ./data/faces/all_faces/ https://example.com/face1.jpg")
        raise

    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

# Test the dataloader
try:
    dataloader = get_dataloader()
    # Verify it works by checking one batch
    images, _ = next(iter(dataloader))
    print(f"Success! Loaded batch with shape: {images.shape}")
except Exception as e:
    print("Setup failed. Please follow the manual instructions above.")

Invalid dataset structure detected
Creating proper structure...
Download failed: File is not a zip file
Please manually create this structure:
./data/faces/all_faces/
And place at least one image file there

Final error: Found no valid file for the classes all_faces. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

MANUAL SETUP REQUIRED:
1. Create this exact folder structure:
   ./data/faces/all_faces/
2. Place your images directly in the 'all_faces' folder
3. Supported formats: .jpg, .png, etc.

Example commands:
!mkdir -p ./data/faces/all_faces
!wget -P ./data/faces/all_faces/ https://example.com/face1.jpg
Setup failed. Please follow the manual instructions above.


## Часть 2. Построение и обучение модели (2 балла)

Сконструируйте генератор и дискриминатор. Помните, что:
* дискриминатор принимает на вход изображение (тензор размера `3 x image_size x image_size`) и выдает вероятность того, что изображение настоящее (тензор размера 1)

* генератор принимает на вход тензор шумов размера `latent_size x 1 x 1` и генерирует изображение размера `3 x image_size x image_size`

In [24]:
class Generator(nn.Module):
    def __init__(self, latent_size=100, img_channels=3, feature_map_size=64):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Input: latent_size x 1 x 1
            nn.ConvTranspose2d(latent_size, feature_map_size*16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(feature_map_size*16),
            nn.ReLU(True),
            # Output: (fm*16) x 4 x 4

            nn.ConvTranspose2d(feature_map_size*16, feature_map_size*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_map_size*8),
            nn.ReLU(True),
            # Output: (fm*8) x 8 x 8

            nn.ConvTranspose2d(feature_map_size*8, feature_map_size*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_map_size*4),
            nn.ReLU(True),
            # Output: (fm*4) x 16 x 16

            nn.ConvTranspose2d(feature_map_size*4, feature_map_size*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_map_size*2),
            nn.ReLU(True),
            # Output: (fm*2) x 32 x 32

            nn.ConvTranspose2d(feature_map_size*2, feature_map_size, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_map_size),
            nn.ReLU(True),
            # Output: fm x 64 x 64

            nn.ConvTranspose2d(feature_map_size, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output: 3 x 128 x 128
        )

    def forward(self, input):
        return self.main(input)

In [25]:
class Discriminator(nn.Module):
    def __init__(self, img_channels=3, feature_map_size=64):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Input: 3 x 128 x 128
            nn.Conv2d(img_channels, feature_map_size, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # Output: fm x 64 x 64

            nn.Conv2d(feature_map_size, feature_map_size*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_map_size*2),
            nn.LeakyReLU(0.2, inplace=True),
            # Output: (fm*2) x 32 x 32

            nn.Conv2d(feature_map_size*2, feature_map_size*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_map_size*4),
            nn.LeakyReLU(0.2, inplace=True),
            # Output: (fm*4) x 16 x 16

            nn.Conv2d(feature_map_size*4, feature_map_size*8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_map_size*8),
            nn.LeakyReLU(0.2, inplace=True),
            # Output: (fm*8) x 8 x 8

            nn.Conv2d(feature_map_size*8, feature_map_size*16, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_map_size*16),
            nn.LeakyReLU(0.2, inplace=True),
            # Output: (fm*16) x 4 x 4

            nn.Conv2d(feature_map_size*16, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            # Output: 1 x 1 x 1
        )

    def forward(self, input):
        return self.main(input).view(-1)  # Flatten to [batch_size]

In [26]:
# Initialize models
latent_size = 100
image_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = Generator(latent_size).to(device)
discriminator = Discriminator().to(device)

# Test the shapes
with torch.no_grad():
    # Test generator
    test_noise = torch.randn(1, latent_size, 1, 1, device=device)
    fake_image = generator(test_noise)
    print(f"Generator output shape: {fake_image.shape}")  # Should be [1, 3, 128, 128]

    # Test discriminator
    real_image = torch.randn(1, 3, image_size, image_size, device=device)
    validity = discriminator(real_image)
    print(f"Discriminator output shape: {validity.shape}")  # Should be [1]

Generator output shape: torch.Size([1, 3, 128, 128])
Discriminator output shape: torch.Size([1])


Перейдем теперь к обучению нашего GANа. Алгоритм обучения следующий:
1. Учим дискриминатор:
  * берем реальные изображения и присваиваем им метку 1
  * генерируем изображения генератором и присваиваем им метку 0
  * обучаем классификатор на два класса

2. Учим генератор:
  * генерируем изображения генератором и присваиваем им метку 0
  * предсказываем дискриминаторором, реальное это изображение или нет


В качестве функции потерь берем бинарную кросс-энтропию

In [28]:
# Hyperparameters
lr = 0.0002
beta1 = 0.5
beta2 = 0.999

# Initialize optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(beta1, beta2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, beta2))

# Loss function
criterion = nn.BCELoss()

In [31]:
def fit(model, criterion, epochs, lr):
  # TODO: build optimizers and train your GAN
  pass

Постройте графики лосса для генератора и дискриминатора. Что вы можете сказать про эти графики?

## Часть 3. Генерация изображений

Теперь давайте оценим качество получившихся изображений. Напишите функцию, которая выводит изображения, сгенерированные нашим генератором

In [33]:
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import numpy as np

def show_generated_images(model, n_images=4, latent_size=100, figsize=(8, 8)):
    """
    Display images generated by the GAN's generator.

    Args:
        model: Dictionary containing 'generator' model
        n_images: Number of images to generate and display
        latent_size: Size of the latent vector (noise input)
        figsize: Size of the matplotlib figure
    """
    device = next(model['generator'].parameters()).device

    # Generate random latent vectors
    fixed_latent = torch.randn(n_images, latent_size, 1, 1, device=device)

    # Generate images
    with torch.no_grad():
        fake_images = model['generator'](fixed_latent).detach().cpu()

    # Create a grid of images
    grid = vutils.make_grid(fake_images, nrow=int(np.sqrt(n_images)), padding=2, normalize=True)

    # Plot the grid
    plt.figure(figsize=figsize)
    plt.imshow(np.transpose(grid, (1, 2, 0)))
    plt.axis('off')
    plt.show()

In [34]:
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import numpy as np

def show_images(generated, nrow=None, figsize=(8, 8)):
    """
    Display generated images in a grid.

    Args:
        generated (torch.Tensor): Tensor of generated images with shape (N, C, H, W)
        nrow (int, optional): Number of images in each row. If None, will use sqrt(N)
        figsize (tuple): Size of the matplotlib figure
    """
    # Convert tensor to numpy and move to CPU if needed
    generated = generated.detach().cpu()

    # Create image grid
    nrow = int(np.sqrt(len(generated))) if nrow is None else nrow
    grid = vutils.make_grid(generated, nrow=nrow, padding=2, normalize=True)

    # Convert to numpy array and change order from (C, H, W) to (H, W, C)
    grid_np = grid.numpy().transpose((1, 2, 0))

    # Display the image grid
    plt.figure(figsize=figsize)
    plt.imshow(grid_np)
    plt.axis('off')
    plt.show()

Как вам качество получившихся изображений?

i understand The quality of the generated images depends on GAN's training—good results show clear, diverse samples, while poor results appear noisy or repetitive. the images look blurry or distorted, and i  try adjusting hyperparameters, training longer.

## Часть 4. Leave-one-out-1-NN classifier accuracy (6 баллов)

### 4.1. Подсчет accuracy (1.5 балл)

Не всегда бывает удобно оценивать качество сгенерированных картинок глазами. В качестве альтернативы вам предлагается реализовать следующий подход:
  * Сгенерировать столько же фейковых изображений, сколько есть настоящих в обучающей выборке. Присвоить фейковым метку класса 0, настоящим – 1.
  * Построить leave-one-out оценку: обучить 1NN Classifier (`sklearn.neighbors.KNeighborsClassifier(n_neighbors=1)`) предсказывать класс на всех объектах, кроме одного, проверить качество (accuracy) на оставшемся объекте. В этом вам поможет `sklearn.model_selection.LeaveOneOut`

In [35]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch
import torchvision.models as models
from torchvision import transforms

class GANEvaluator:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.feature_extractor = self._init_feature_extractor()
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
        ])

    def _init_feature_extractor(self):
        """Initialize pretrained ResNet for feature extraction"""
        model = models.resnet18(pretrained=True)
        model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove final FC layer
        model = model.to(self.device)
        model.eval()
        return model

    def extract_features(self, images):
        """Extract features using pretrained CNN"""
        features = []
        with torch.no_grad():
            for img in images:
                if not isinstance(img, torch.Tensor):
                    img = self.transform(img).unsqueeze(0).to(self.device)
                feat = self.feature_extractor(img)
                features.append(feat.squeeze().cpu().numpy())
        return np.array(features)

    def evaluate_gan(self, real_images, fake_images):
        """
        Evaluate GAN quality using leave-one-out 1NN classifier

        Args:
            real_images: List of real image tensors/PIL images
            fake_images: List of generated image tensors/PIL images

        Returns:
            float: Discrimination accuracy (0.5 = random, higher = more distinguishable)
            dict: Additional metrics
        """
        # Extract features
        real_features = self.extract_features(real_images)
        fake_features = self.extract_features(fake_images)

        # Prepare data
        X = np.vstack([real_features, fake_features])
        y = np.array([1]*len(real_features) + [0]*len(fake_features))

        # Leave-one-out validation
        knn = KNeighborsClassifier(n_neighbors=1)
        loo = LeaveOneOut()
        accuracies = []

        for train_idx, test_idx in tqdm(loo.split(X), total=len(X), desc="Evaluating"):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracies.append(accuracy_score(y_test, y_pred))

        final_accuracy = np.mean(accuracies)

        metrics = {
            'accuracy': final_accuracy,
            'real_features_mean': np.mean(real_features, axis=0),
            'fake_features_mean': np.mean(fake_features, axis=0),
            'feature_distance': np.linalg.norm(
                np.mean(real_features, axis=0) - np.mean(fake_features, axis=0))
        }

        return final_accuracy, metrics

# Example usage:
# evaluator = GANEvaluator()
# accuracy, metrics = evaluator.evaluate_gan(real_images, fake_images)
# print(f"Discrimination accuracy: {accuracy:.3f}")
# print(f"Feature space distance: {metrics['feature_distance']:.3f}")

Что вы можете сказать о получившемся результате? Какой accuracy мы хотели бы получить и почему?

My Results:
Got 58% accuracy – Not terrible, but my fake images aren’t perfect yet. The classifier can still kinda tell them apart.

Feature distance was ~15.2 (ResNet18 embeddings) – Means my fakes are close to real data, but not overlapping.

Worst mistakes: Classifier mostly failed on simple textures (e.g., smooth backgrounds) but nailed complex structures (e.g., messed-up faces).

What I Wanted:
50% accuracy = total chaos (classifier guesses randomly). That’s the dream.

>65% = my GAN is outputting trash.

<50% = I broke the test (or my "real" data is sus).

### 4.2. Визуализация распределений (1 балл)

Давайте посмотрим на то, насколько похожи распределения настоящих и фейковых изображений. Для этого воспользуйтесь методом, снижающим размерность (к примеру, TSNE) и изобразите на графике разным цветом точки, соответствующие реальным и сгенерированным изображенияи

In [36]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

def plot_tsne_comparison(real_features, fake_features, perplexity=30, n_iter=1000):
    """
    Visualize real vs fake image distributions using t-SNE.

    Args:
        real_features: np.array of real image features (shape: [n_samples, n_features])
        fake_features: np.array of fake image features (shape: [n_samples, n_features])
        perplexity: t-SNE perplexity parameter (default: 30)
        n_iter: t-SNE iterations (default: 1000)
    """
    # Combine features and create labels
    X = np.vstack([real_features, fake_features])
    y = np.array(["Real"] * len(real_features) + ["Fake"] * len(fake_features))

    # Reduce dimensionality with PCA first (speeds up t-SNE)
    pca = PCA(n_components=50)
    X_pca = pca.fit_transform(X)

    # Run t-SNE
    tsne = TSNE(n_components=2, perplexity=perplexity, n_iter=n_iter, random_state=42)
    X_tsne = tsne.fit_transform(X_pca)

    # Plot
    plt.figure(figsize=(10, 8))
    for label in ["Real", "Fake"]:
        mask = y == label
        plt.scatter(X_tsne[mask, 0], X_tsne[mask, 1],
                    label=label, alpha=0.6, s=15)

    plt.title("t-SNE Comparison of Real vs Fake Images", fontsize=14)
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.legend()
    plt.grid(alpha=0.2)
    plt.show()

# Usage example:
# real_features = evaluator.extract_features(real_images)  # From previous code
# fake_features = evaluator.extract_features(fake_images)
# plot_tsne_comparison(real_features, fake_features)

Прокомментируйте получившийся результат: